In [1]:
import pickle

fname = '/home/anton/Documents/Tulane/Hackathon/ImageNomeR/data/fmri-FC-slim.pkl'
fmriDict = None

with open(fname, 'rb') as f:
    fmriDict = pickle.load(f)
    
print(fmriDict.keys())

dict_keys(['FC-slim', 'subjNum2IdxMap', 'subjIdx2NumMap', 'groupsNormalDiagMap'])


In [6]:
def getGroupIdcs(groupsMap):
    normals = []
    fibros = []
    for num,diag in groupsMap.items():
        if diag == 1:
            normals.append(num)
        else:
            fibros.append(num)
    return normals,fibros

normals, fibros = getGroupIdcs(fmriDict['groupsNormalDiagMap'])

print(len(normals))
print(normals)
print(len(fibros))
print(fibros)

33
['007', '012', '014', '016', '018', '021', '022', '026', '030', '031', '032', '033', '034', '036', '042', '045', '047', '056', '058', '059', '060', '061', '064', '066', '068', '069', '070', '072', '073', '074', '075', '076', '077']
33
['002', '004', '005', '006', '008', '009', '010', '011', '013', '015', '017', '019', '020', '023', '024', '025', '028', '029', '037', '038', '039', '040', '043', '044', '046', '049', '050', '052', '053', '054', '055', '062', '063']


In [8]:
print(fmriDict['FC-slim'].shape)

(68, 34716)


In [31]:
import random
import numpy as np

FCslim = fmriDict['FC-slim']
subjNum2Idx = fmriDict['subjNum2IdxMap']

random.shuffle(normals)
random.shuffle(fibros)

normalTrain = normals[:25]
normalTest = normals[25:]

fibrosTrain = fibros[:25]
fibrosTest = fibros[25:]

train = normalTrain + fibrosTrain
test = normalTest + fibrosTest

Xtrain = [FCslim[subjNum2Idx[num]] for num in train]
Xtest = [FCslim[subjNum2Idx[num]] for num in test]

Xtrain = np.stack(Xtrain)
Xtest = np.stack(Xtest)

ytrain = np.concatenate([np.ones(25), np.zeros(25)])
ytest = np.concatenate([np.ones(8), np.zeros(8)])

print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(50, 34716)
(16, 34716)
(50,)
(16,)


In [32]:
# Run a simple analysis

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000).fit(Xtrain, ytrain)
yhat = clf.predict(Xtest)
print(yhat)
print(sum(yhat == ytest)/len(ytest))

[0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1.]
0.625


In [33]:
a,b = np.triu_indices(264,1)
idcs = np.arange(34716)

labels = [f'{a[i]}-{b[i]}' for i in idcs]

print(labels[0:10])
print(labels[-10:])

['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10']
['259-260', '259-261', '259-262', '259-263', '260-261', '260-262', '260-263', '261-262', '261-263', '262-263']


In [34]:
import json

jsonCompare = 'normal(1) vs. fibromyalgia(0)'
jsonAccuracy = sum(yhat == ytest)/len(ytest)
jsonTrain = [25,25]
jsonTest = [8,8]
jsonWeights = clf.coef_.squeeze()
# jsonWeights = base64.b64encode(jsonWeights)
jsonLabels = labels

jsonObj = {
    'Compare': jsonCompare,
    'Model': 'Logistic Regression',
    'Accuracy': jsonAccuracy,
    'Train': jsonTrain,
    'Test': jsonTest,
    'Weights': list(jsonWeights),
    'Labels': jsonLabels
}

print(len(json.dumps(jsonObj)))

1143007


In [37]:
import pickle

jsonFilename = '/home/anton/Documents/Tulane/Hackathon/ImageNomeR/data/dummyFmri1.json'

with open(jsonFilename, 'w') as f:
    json.dump(jsonObj, f)
    
print('Complete')

Complete
